<a href="https://colab.research.google.com/github/vranade-cyber/11785_Part-2/blob/main/DL1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#These libraries help to interact with the operating system and the runtime environment respectively
import os
import sys
import time

#Model/Training related libraries
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#Dataloader libraries
from torch.utils.data import DataLoader, Dataset


Readme is part of code as comments


In [ ]:
#Intall Kaggle API and create kaggle directory
!pip install kaggle
!mkdir .kaggle
!pip install -q kaggle
#This data is used to login  into your Kaggle account
import json
token = {"username":"virajranade","key":"152c772ecb15eebcb68eb988aaf4e2fd"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
! chmod 600 ~/.kaggle/kaggle.json

!chmod 600 /content/.kaggle/kaggle.json
!cp /content/.kaggle/kaggle.json /root/.kaggle/
!kaggle config set -n path -v /content

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
- path is now set to: /content


In [ ]:
#!kaggle competitions download -c hw1p2-toy-problem
!kaggle competitions download -c idl-fall2021-hw1p2

 98% 237M/241M [00:08<00:00, 20.8MB/s]
100% 241M/241M [00:08<00:00, 31.1MB/s]
  0% 0.00/5.16M [00:00<?, ?B/s]
100% 5.16M/5.16M [00:00<00:00, 171MB/s]
 99% 1.90G/1.92G [00:49<00:00, 38.0MB/s]
100% 1.92G/1.92G [00:49<00:00, 41.5MB/s]
  0% 0.00/617k [00:00<?, ?B/s]
100% 617k/617k [00:00<00:00, 88.5MB/s]
 97% 238M/246M [00:01<00:00, 229MB/s]
100% 246M/246M [00:01<00:00, 242MB/s]
  0% 0.00/4.03M [00:00<?, ?B/s]
100% 4.03M/4.03M [00:00<00:00, 66.2MB/s]


In [ ]:
!unzip competitions/idl-fall2021-hw1p2/train.npy.zip
!unzip competitions/idl-fall2021-hw1p2/train_labels.npy.zip
!unzip competitions/idl-fall2021-hw1p2/dev.npy.zip
!unzip competitions/idl-fall2021-hw1p2/dev_labels.npy.zip
!unzip competitions/idl-fall2021-hw1p2/test.npy.zip


Archive:  competitions/idl-fall2021-hw1p2/train.npy.zip
  inflating: train.npy               
Archive:  competitions/idl-fall2021-hw1p2/train_labels.npy.zip
  inflating: train_labels.npy        
Archive:  competitions/idl-fall2021-hw1p2/dev.npy.zip
  inflating: dev.npy                 
Archive:  competitions/idl-fall2021-hw1p2/dev_labels.npy.zip
  inflating: dev_labels.npy          
Archive:  competitions/idl-fall2021-hw1p2/test.npy.zip
  inflating: test.npy                


Here we have used vstack to create the dataset.

In [ ]:
train_data = np.vstack(np.load('train.npy',allow_pickle=True))
print(np.shape(train_data))


(18482968, 40)


Similarly for val and test data

In [ ]:
test_data = np.vstack(np.load('test.npy',allow_pickle=True))
print(np.shape(test_data))
val_data = np.vstack(np.load('dev.npy',allow_pickle=True))
print(np.shape(val_data))


(1910012, 40)
(1935669, 40)


To create a set of empty labels for test_labels.
We use test_set and create it.

In [ ]:
k=np.shape(test_data)[0]
test_labels=[]

for i in range(k):
  c=np.shape(test_data[i])[0]
  h=np.ones((c,))
  test_labels.append(h)
np.array(test_labels,dtype=object)
print(np.shape(test_labels))

(1910012, 40)


In [ ]:
train_labels=np.concatenate(np.load('train_labels.npy',allow_pickle=True))
val_labels=np.concatenate(np.load('dev_labels.npy',allow_pickle=True))
print(np.shape(val_data))
print(np.shape(val_labels))

(1935669, 40)
(1935669,)


In [ ]:
class MyMLPDataset(Dataset):
  def __init__(self,X,y,context=0):
    self.data=X
    self.labels=y
    assert len(self.data)==len(self.labels)
    self.length=len(self.labels)

    self.context=context
    self.data=np.pad(self.data,((self.context,self.context),(0,0)),mode='constant', constant_values=0).astype('float32')


  def __len__(self):
    return self.length

  def __getitem__(self,index):

    x=self.data[index:(index+(2*self.context)+1)].flatten()
    y=self.labels[index].astype(np.long)


    return x,y

In [ ]:
## Dataloaders

# Training dataloader
train_dataset = MyMLPDataset(train_data,train_labels,context=20)
train_args = dict(shuffle = True, batch_size = 128, num_workers=2)
train_loader = DataLoader(train_dataset, **train_args)

In [ ]:
# Validation dataloader
val_dataset = MyMLPDataset(val_data, val_labels,context=20)
val_args = dict(shuffle = False, batch_size = 128, num_workers=2)
val_loader = DataLoader(val_dataset, **val_args)
#collate_fn=MLPDataset.collate_fn-> Use this while using Index map dataset.
### Slower but accuracy is better.

#Test dataLoader

test_dataset = MyMLPDataset(test_data, test_labels,context=20)
test_args = dict(shuffle = False, batch_size = 128, num_workers=2)
test_loader = DataLoader(test_dataset, **test_args)

In [ ]:
for batch, (data, targets) in enumerate(test_loader):
  print (data.shape)
  print (targets.shape)
  break

torch.Size([128, 1640])
torch.Size([128, 40])


In [ ]:
for batch,(data,targets) in enumerate(train_loader):
   print (data.shape)
   print (targets.shape)
   if (batch > 4):
     break

torch.Size([128, 1640])
torch.Size([128])
torch.Size([128, 1640])
torch.Size([128])
torch.Size([128, 1640])
torch.Size([128])
torch.Size([128, 1640])
torch.Size([128])
torch.Size([128, 1640])
torch.Size([128])
torch.Size([128, 1640])
torch.Size([128])


1st attempt was same Model, except with ReLU
After 3 Epochs I got Acc 75.1
I needed a stronger model so I trained this

In [ ]:
## Model Architecture definition

class MLP(nn.Module):

    # define model elements
    def __init__(self, size):
        super(MLP, self).__init__()
        
        # Sequential model definition: Input -> Linear -> ReLU -> Linear -> Output
        Linear_layers=[
                       nn.Linear(1640,4608,True),nn.BatchNorm1d(4608,eps=1e-08),nn.LeakyReLU(),
                       nn.Linear(4608,4608,True),nn.BatchNorm1d(4608,eps=1e-08),nn.LeakyReLU(),
                       nn.Linear(4608,4608,True),nn.BatchNorm1d(4608,eps=1e-08),nn.LeakyReLU(),
                       nn.Linear(4608,2304,True),nn.BatchNorm1d(2304,eps=1e-08),nn.LeakyReLU(),
                       nn.Linear(2304,1152,True),nn.BatchNorm1d(1152,eps=1e-08),nn.LeakyReLU(),
                       nn.Linear(1152,576,True),nn.BatchNorm1d(576,eps=1e-08),nn.LeakyReLU(),
                       nn.Linear(576,72,True)]
        self.layers = nn.Sequential(*Linear_layers)

    def forward(self, x):
        #torch.save(model.state_dict(), 'C:/Users/ADMIN/Desktop/Fall_21/11785/Homeworks/hw1p1')
        # Model forward pass
        return self.layers(x)

In [ ]:
# Model
model = MLP([440,9216,4608,2304,1152,576,72])
#model = MLP([32,440])

# Define Criterion/ Loss function
criterion = nn.CrossEntropyLoss()

# Define Adam Optimizer

## refer to TA

device=torch.device('cuda' if torch.cuda else 'cpu')
#device=torch.device('cpu')
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,[5,10])
criterion=  nn.CrossEntropyLoss()
model=model.to(device)
print(model)

MLP(
  (layers): Sequential(
    (0): Linear(in_features=1640, out_features=4608, bias=True)
    (1): BatchNorm1d(4608, eps=1e-08, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): Linear(in_features=4608, out_features=4608, bias=True)
    (4): BatchNorm1d(4608, eps=1e-08, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Linear(in_features=4608, out_features=4608, bias=True)
    (7): BatchNorm1d(4608, eps=1e-08, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.01)
    (9): Linear(in_features=4608, out_features=2304, bias=True)
    (10): BatchNorm1d(2304, eps=1e-08, momentum=0.1, affine=True, track_running_stats=True)
    (11): LeakyReLU(negative_slope=0.01)
    (12): Linear(in_features=2304, out_features=1152, bias=True)
    (13): BatchNorm1d(1152, eps=1e-08, momentum=0.1, affine=True, track_running_stats=True)
    (14): LeakyReLU(negative_slo

In case the saved model needs to be Loaded

In [ ]:
## Load the model for testing 
## Older Model
best_model_loc = 'drive/MyDrive/model.pth'
checkpoint = torch.load(best_model_loc)
model.load_state_dict(checkpoint['model_state_dict'])
print(checkpoint['epoch'])
## Do not Run

In [ ]:
# Train the model

def train_model(train_loader, model,criterion,optimizer,scheduler):

  training_loss = 0
  
  # Set model in 'Training mode'
  model.train()
  
  # enumerate mini batches
  for batch, (data,targets) in enumerate(train_loader):
      
      # clear the gradients
      optimizer.zero_grad()
      data=data.to(device)
      targets=targets.to(device)
      #mode=model.to(device)
      
      # compute the model output
      output = model(data)
      
      # calculate loss
      loss = criterion(output,targets)
      
      # Backward pass
      loss.backward()
      
      # Update model weights
      optimizer.step()

      training_loss += loss.item()
      
      #end_time=time.time()
      #print('Time for 1 iteration is',end_time-start_time)
      #start_time=end_time
  training_loss /= len(train_loader)
  scheduler.step()
  return training_loss

In [ ]:
def val_model(val_loader,model,criterion,optimizer):
  size=len(val_loader.dataset)
  num_batches=len(val_loader)
  test_loss=0
  correct=0
  model.eval()

  with torch.no_grad():
    for index,(data,labels) in enumerate(val_loader):
      data=data.to(device)
      labels=labels.to(device)

      pred=model(data)
      predicted_labels=pred.argmax(1)
      loss=criterion(pred,labels)
      test_loss +=loss.item()
      correct +=(pred.argmax(1)==labels).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size

    print(f'\n Test Error is ',correct*100,'Accuracy', test_loss,'Average Loss')
    return (correct,test_loss)


By the time I tested, I had only enough time for 1 epoch.
Thus I ran only 1 epoch

In [ ]:
# Define number of epochs
nepochs = 1
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,[5,10])
#CUDA_LAUNCH_BLOCKING=1

for epoch in range(nepochs):
    #model.train()

    # Train
    epoch_start_time=time.time()

    training_loss = train_model(train_loader,model,criterion,optimizer,scheduler)
    val_acc, val_loss = val_model(val_loader, model,criterion,optimizer)
    # enumerate mini batches
    
    '''
    for i, (data,targets) in enumerate(train_loader):
        
        # clear the gradients
        #optimizer.zero_grad()
        #data=data.to(device)
        #targets=targets.to(device)
        # compute the model output
        output = model(data)
        # calculate loss
        loss = criterion(output,targets)
        
        # Backward pass
        loss.backward()

        # scheduler
        
        #torch.save(model.state_dict(), 'model_weights.pth')
        # Print log of accuracy and loss
        '''
    epoch_end_time=time.time()
    #model = model.vgg16(pretrained=True)
    filename = str(f"mymodel_epoch_{epoch}.pth")
    torch.save({'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict' : scheduler.state_dict()}, filename)
    

    print("Epoch: "+str(epoch)+", Training loss: "+str(training_loss),'Time required',(epoch_end_time-epoch_start_time))
    print("Epoch: "+str(epoch)+", Val_accuracy: "+str(val_acc)+", Validation loss:"+str(val_loss)+
          ", Validation accuracy:"+str(val_acc*100)+"%")


 Test Error is  74.962816473271 Accuracy 0.8356455934455601 Average Loss
Epoch: 0, Training loss: 0.760509930688249 Time required 3097.6073050498962
Epoch: 0, Val_accuracy: 0.74962816473271, Validation loss:0.8356455934455601, Validation accuracy:74.962816473271%


In [ ]:
def test_model(test_loader,model):
  #size=len(test_loader.dataset)
  #num_batches=len(test_loader)
  #test_loss=0
  #correct=0
  model.eval()
  predictions=[]
  with torch.no_grad():
    for index,(X,y) in enumerate(test_loader):
      X=X.to(device)
      #y=y.to(device)

      predicted_labels=model(X)
      #predicted_labels=pred.argmax(1)
      #loss=criterion(pred,y)
      #test_loss +=loss.item()
      #correct +=(pred.argmax(1)==y).type(torch.float).sum().item()
      predicted_labels=predicted_labels.detach()
      predicted_labels=predicted_labels.cpu()
      predicted_labels=predicted_labels.numpy()
      predicted_labels = np.argmax(predicted_labels, axis=1)
      predicted_labels = np.reshape(predicted_labels,(-1,1))
      predictions.append(predicted_labels)
    #test_loss /= num_batches
    #correct /= size
    
    # predictions=np.vstack(predictions)
    #print(f'\n Test Error is ',correct*100,'Accuracy', test_loss,'Average Loss')
    return (predictions)

In [ ]:
best_model_loc = '/content/mymodel_epoch_0.pth'
checkpoint = torch.load(best_model_loc)
model.load_state_dict(checkpoint['model_state_dict'])
print(checkpoint['epoch'])

0


In [ ]:
pred = test_model(test_loader, model)
print(len(pred))
predictions = list(np.concatenate(pred).flat)
ids = np.arange(len(predictions))
len(ids)

14922


1910012

In [ ]:
import pandas as pd
my_submission_2=pd.DataFrame({'id':ids,'label':predictions})
my_submission_2.to_csv('try_7.csv', index=False)
!kaggle competitions submit -c idl-fall2021-hw1p2 -f 'try_6.csv' -m 'try7'

100% 18.5M/18.5M [00:10<00:00, 1.83MB/s]
Successfully submitted to IDL-Fall21-HW1P2

In [ ]:
from google.colab import files
files.download('try_2.csv') 